In [1]:
!pip install nilearn
import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
import nibabel as nib
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.ensemble import StackingClassifier

brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
crad = datasets.fetch_atlas_craddock_2012()

     |████████████████████████████████| 10.0 MB 4.9 MB/s 


/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)



Dataset created in /root/nilearn_data/oasis1



Downloaded 867459072 of 905208634 bytes (95.8%,    0.5s remaining) ...done. (12 seconds, 0 min)
Extracting data from /root/nilearn_data/oasis1/adcb228c7b26ed4f4fba93bd32bf8d97/archive_dartel.tgz..... done.
/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:2349: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)



Dataset created in /root/nilearn_data/craddock_2012



 ...done. (1 seconds, 0 min)
Extracting data from /root/nilearn_data/craddock_2012/45a4e574a5c116b8be81d080bdaa0814/craddock_2011_parcellations.tar.gz..... done.


In [4]:
def get_base_model(i):
  atlas_nii = index_img(crad['scorr_mean'], i) # this needs to be modified for some tasks
  print('%i regions found in this atlas' % (len(np.unique(atlas_nii.get_data().ravel())) - 1))
  masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
  input_variables = masker.fit_transform(brain_data.gray_matter_maps)
  output_variable = np.array(brain_data.ext_vars.mf == b'F', dtype=np.int) # gives 1 for females and 0 for males

  ############## note no bootstrap, just taking the first 80% of 100 subjects, that is the first 80 subjects as training set ############

  train_input = input_variables[:80]
  train_output = output_variable[:80]
  test_input = input_variables[80:]
  test_output = output_variable[80:]
  
  model = LogisticRegression(C=1.0).fit(train_input,train_output)

  train_proba = model.predict_proba(train_input)
  test_proba = model.predict_proba(test_input)

  train_single = []
  test_single = []

  for train in train_proba:
    if train[0] >= train[1]:
      train_single.append(train[0])
    else:
      train_single.append(train[1])

  for test in test_proba:
    if test[0] >= test[1]:
      test_single.append(test[0])
    else:
      test_single.append(test[1])

  return model, test_single, train_single
  # return model, test_proba, train_proba


#### the output labels should be the same for any number of areas of interests. So we just pick randomly a 1000 one #########

atlas_nii = index_img(crad['scorr_mean'], 42) # this needs to be modified for some tasks
print('%i regions found in this atlas' % (len(np.unique(atlas_nii.get_data().ravel())) - 1))
masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = np.array(brain_data.ext_vars.mf == b'F', dtype=np.int) # gives 1 for females and 0 for males


train_input = input_variables[:80]
train_output = output_variable[:80]
test_input = input_variables[80:]
test_output = output_variable[80:]

estimators = []  
train_probas = []
test_probas = []
for i in range (0,43):
  model, test_proba, train_proba = get_base_model(i)
  estimators.append((str(i),model))
  test_probas.append(test_proba)
  train_probas.append(train_proba)

train_probas_flat = [item for sublist in train_probas for item in sublist]
test_probas_flat = [item for sublist in test_probas for item in sublist]

train_probas_flat = np.reshape(train_probas_flat,(-1,1))
test_probas_flat = np.reshape(test_probas_flat,(-1,1))

train_output_extend = list(train_output)*43
test_output_extend = list(test_output)*43

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(),stack_method='predict_proba')

clf.fit(train_probas_flat,train_output_extend)

print("Accracy after stacking:")

clf.score(test_probas_flat, test_output_extend)

# clf.fit(train_input,train_output).score(test_input, test_output)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


788 regions found in this atlas


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  This is separate from the ipykernel package so we can avoid doing imports until


10 regions found in this atlas


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  This is separate from the ipykernel package so we can avoid doing imports until


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  This is separate from the ipykernel package so we can avoid doing imports until


740 regions found in this atlas


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  This is separate from the ipykernel package so we can avoid doing imports until


788 regions found in this atlas
Accracy after stacking:


0.48023255813953486